In [0]:
%sql
select * from datamodeling.silver.silver_table;

order_id,order_date,customer_id,customer_name,customer_email,product_id,product_name,product_category,quantity,unit_price,payment_type,country,last_updated,upper_customer_name,processDate
1002,2024-07-01,2,Bob Smith,bob@yahoo.com,502,Airpods pro,Electronics,2,199.99,PayPal,USA,2024-07-01,BOB SMITH,2026-02-05
1003,2024-07-01,3,Charlie Brown,charlie@hotmail.com,503,Nike shoes,Foot Wear,1,129.99,Credit Card,Canada,2024-07-01,CHARLIE BROWN,2026-02-05
1001,2024-07-01,1,Alice Johnson,alice@gmailcom,501,iphone 14,Electronics,1,999.99,Credit Card,USA,2024-07-01,ALICE JOHNSON,2026-02-05
1004,2024-07-02,4,David Kim,david@gmail.com,504,Adidas Shoes,Foot Wear,1,149.99,PayPal,Canada,2024-07-02,DAVID KIM,2026-02-05
1005,2024-07-02,5,Eve Lee,eve@hotmail.com,505,Samsung TV,Electronics,1,799.99,Credit Card,USA,2024-07-02,EVE LEE,2026-02-05


### surrogate key are unique system generated key which are integers they are stable and efficient keys.
 

In [0]:
%sql
create table datamodeling.gold.DimCustomers
AS 
select *,row_number() over(order by customer_id) as DimCustomerKey from(
    select distinct(customer_id),customer_name,customer_email,upper_customer_name from datamodeling.silver.silver_table
)a

num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from datamodeling.gold.dimcustomers;

customer_id,customer_name,customer_email,upper_customer_name,DimCustomerKey
1,Alice Johnson,alice@gmailcom,ALICE JOHNSON,1
2,Bob Smith,bob@yahoo.com,BOB SMITH,2
3,Charlie Brown,charlie@hotmail.com,CHARLIE BROWN,3
4,David Kim,david@gmail.com,DAVID KIM,4
5,Eve Lee,eve@hotmail.com,EVE LEE,5


###### DimProducts

In [0]:
%sql
create table datamodeling.gold.DimProducts
as
with cte as(
select distinct(product_id),product_name,product_category from datamodeling.silver.silver_table
)
select *,row_number() over(order by product_id) as DimProductKey from cte;

num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from datamodeling.gold.dimproducts;

product_id,product_name,product_category,DimProductKey
501,iphone 14,Electronics,1
502,Airpods pro,Electronics,2
503,Nike shoes,Foot Wear,3
504,Adidas Shoes,Foot Wear,4
505,Samsung TV,Electronics,5


In [0]:
%sql
create table datamodeling.gold.dimpayments
as
with cte as(
  select distinct(payment_type) from datamodeling.silver.silver_table
)
select *,row_number() over(order by payment_type) as DimPaymentKey from cte;

num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from datamodeling.gold.dimpayments;

payment_type,DimPaymentKey
Credit Card,1
PayPal,2


#####DimRegions

In [0]:
%sql
create table datamodeling.gold.dimRegions
as
with cte as(
  select distinct(country) from datamodeling.silver.silver_table
)
select *,row_number() over(order by country) as DimRegionKey from cte

num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from datamodeling.gold.dimRegions;

country,DimRegionKey
Canada,1
USA,2


#####DimSales


In [0]:
%sql
create table datamodeling.gold.dimSales
as
select 
row_number() over(order by order_id) as DimSalesKey,
 order_id,
 order_date,
 customer_id,
 customer_name,
 customer_email,
 product_id,
 product_name,
 product_category,
 payment_type,
 country,
 last_updated,
 upper_customer_name,
 processDate
 from datamodeling.silver.silver_table;

num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from datamodeling.gold.dimSales;

DimSalesKey,order_id,order_date,customer_id,customer_name,customer_email,product_id,product_name,product_category,payment_type,country,last_updated,upper_customer_name,processDate
1,1001,2024-07-01,1,Alice Johnson,alice@gmailcom,501,iphone 14,Electronics,Credit Card,USA,2024-07-01,ALICE JOHNSON,2026-02-05
2,1002,2024-07-01,2,Bob Smith,bob@yahoo.com,502,Airpods pro,Electronics,PayPal,USA,2024-07-01,BOB SMITH,2026-02-05
3,1003,2024-07-01,3,Charlie Brown,charlie@hotmail.com,503,Nike shoes,Foot Wear,Credit Card,Canada,2024-07-01,CHARLIE BROWN,2026-02-05
4,1004,2024-07-02,4,David Kim,david@gmail.com,504,Adidas Shoes,Foot Wear,PayPal,Canada,2024-07-02,DAVID KIM,2026-02-05
5,1005,2024-07-02,5,Eve Lee,eve@hotmail.com,505,Samsung TV,Electronics,Credit Card,USA,2024-07-02,EVE LEE,2026-02-05


####Fact Table

In [0]:
%sql
create table datamodeling.gold.factSales
as
select 
  D.DimSalesKey,
  S.DimCustomerKey,
  P.DimProductKey,
  T.DimPaymentKey,
  R.DimRegionKey,
  F.quantity,
  F.unit_price
from
  datamodeling.silver.silver_table as F
left join datamodeling.gold.dimcustomers as S
on F.customer_id = S.customer_id
left join datamodeling.gold.dimproducts as P
on F.product_id = P.product_id
left join datamodeling.gold.dimpayments as T
on F.payment_type = T.payment_type
left join datamodeling.gold.dimRegions as R
on F.country = R.country
left join datamodeling.gold.dimSales as D
on F.order_id = D.order_id;


num_affected_rows,num_inserted_rows


In [0]:
%sql
select * from datamodeling.gold.factSales;

DimSalesKey,DimCustomerKey,DimProductKey,DimPaymentKey,DimRegionKey,quantity,unit_price
2,2,2,2,2,2,199.99
3,3,3,1,1,1,129.99
1,1,1,1,2,1,999.99
4,4,4,2,1,1,149.99
5,5,5,1,2,1,799.99
